In [51]:
import pandas as pd
import numpy as np
import pickle
import locale 
import datetime
locale.setlocale(locale.LC_TIME, 'es_ES.utf8')


'es_ES.utf8'

In [52]:
from pandas.api.types import is_datetime64_any_dtype as is_datetime

In [54]:
df = pd.read_excel("ensayo.xlsx")

In [66]:
fsiembra

Timestamp('2019-03-07 00:00:00')

[Timestamp('2021-12-09 00:00:00'),
 Timestamp('2024-09-12 00:00:00'),
 Timestamp('2027-06-17 00:00:00')]

In [61]:
type(fsiembra)

pandas._libs.tslibs.timestamps.Timestamp

In [ ]:
[fsiembra]

In [19]:
columnas = [x for x in df.columns if x.startswith("abs")]
bloque_seleccionado = df.query("bloque==210719")[columnas]


In [43]:
def crear_df_trimestral_transpuesto (df,trimestre):
    trim = str(trimestre)
    columnas = [x for x in df.columns if "q"+trim in x]
    df_trim = df[columnas]
    df_trim.columns = [x[4:-3] for x in df_trim.columns]
    df_trim= df_trim.transpose()
    df_trim.columns=["q"+trim]
    return df_trim

In [47]:
df_q1 = crear_df_trimestral_transpuesto(bloque_seleccionado,1)
df_q2 = crear_df_trimestral_transpuesto(bloque_seleccionado,2)
df_q3 = crear_df_trimestral_transpuesto(bloque_seleccionado,3)
df_q4 = crear_df_trimestral_transpuesto(bloque_seleccionado,4)

In [50]:
df_q1.merge(df_q2,left_index=True,right_index=True)

,q1,q2
menos_de_10,0,0
mas_de_20,1,2
conteo,2,6


In [35]:
df_q1.transpose().columns=["q1"]

In [30]:
df_q1.columns[0][4:-3]

'menos_de_10'

In [13]:
fill_value = (df["apldate"]- df["finiciosiembra"])/np.timedelta64(1, 'D')

In [15]:
df["diff2"] = df["diff"]

In [19]:
df["diff2"].head()

0     NaN
1    14.0
2    31.0
3    11.0
4    13.0
Name: diff2, dtype: float64

In [21]:
df["diff"]=df["diff"].fillna(fill_value)

In [11]:
#df["tooltip"][0]

In [9]:
df["tooltip"] = df.apply(lambda row: f"aplicaciones con más de 20 días de diferencia: {row['porc diff menos de 10 q1']} \n aplicaciones con menos de 10 días de diferencia {row['porc diff más de 20 q3']} ",axis=1)

In [28]:
tooltip_data=[
        {
            row["bloque"]: {'value': str(row["q1"]), 'type': 'markdown'}
        } for row in df[["bloque","q1"]].to_dict('records')
    ]

In [29]:
tooltip_data

[{223519: {'value': '0.63', 'type': 'markdown'}},
 {224819: {'value': '0.63', 'type': 'markdown'}},
 {225319: {'value': '0.63', 'type': 'markdown'}},
 {225419: {'value': '0.63', 'type': 'markdown'}},
 {225519: {'value': '0.63', 'type': 'markdown'}}]

In [18]:
tooltip_data

[{'bloque': {'value': '223519', 'type': 'markdown'},
  'q1': {'value': '0.63', 'type': 'markdown'}},
 {'bloque': {'value': '224819', 'type': 'markdown'},
  'q1': {'value': '0.63', 'type': 'markdown'}},
 {'bloque': {'value': '225319', 'type': 'markdown'},
  'q1': {'value': '0.63', 'type': 'markdown'}},
 {'bloque': {'value': '225419', 'type': 'markdown'},
  'q1': {'value': '0.63', 'type': 'markdown'}},
 {'bloque': {'value': '225519', 'type': 'markdown'},
  'q1': {'value': '0.63', 'type': 'markdown'}}]

In [29]:
df[(df['apldate'] < df['finiciosiembra'] + datetime.timedelta(weeks=12)) ].groupby("blocknumber")["apldate"].agg(num_apls_q1="count").reset_index()

,num_apls_q1
blocknumber,
210719,2
210819,2
211219,3
211419,3


In [31]:
df[(df['apldate'] > df['finiciosiembra'] + datetime.timedelta(weeks=12)) & (df['apldate'] < df['finiciosiembra'] + datetime.timedelta(weeks=24)) ].groupby("blocknumber")["apldate"].agg(num_apls_q2="count").reset_index()

,num_apls_q1
blocknumber,
210719,5
210819,5
211219,5
211419,5


In [32]:
df[(df['apldate'] > df['finiciosiembra'] + datetime.timedelta(weeks=24)) & (df['apldate'] < df['finiciosiembra'] + datetime.timedelta(weeks=36)) ].groupby("blocknumber")["apldate"].agg(num_apls_q3="count").reset_index()

,num_apls_q3
blocknumber,
210719,4
210819,4
211219,4
211419,4


In [34]:
df[(df['apldate'] > df['finiciosiembra'] + datetime.timedelta(weeks=36))].groupby("blocknumber")["apldate"].agg(num_apls_q4="count").reset_index()

,blocknumber,num_apls_q3
0,210719,3
1,210819,3
2,211219,3
3,211419,3


In [8]:
df.query("apldate>finiciosiembra")

,Unnamed: 0,codigo,formula,apldate,blocknumber,plantcant,finiciosiembra,finduccion,descripcion,lote,area,gruposiembra,descripcion_formula,categoria,diff
0,1,3621,003,2019-04-09,210719,32197,2019-02-28,2020-03-27,210719,21,5446,256,FOLIAR 1,Fertilizante,NaN
1,2,3668,004,2019-04-23,210719,32197,2019-02-28,2020-03-27,210719,21,5446,256,FOLIAR 2,Fertilizante,14.0
2,5,3773,007,2019-05-24,210719,32197,2019-02-28,2020-03-27,210719,21,5446,256,FOLIAR 3,Fertilizante,31.0
3,7,3791,008,2019-06-04,210719,32197,2019-02-28,2020-03-27,210719,21,5446,256,FOLIAR 4,Fertilizante,11.0
4,8,3839,011,2019-06-17,210719,32197,2019-02-28,2020-03-27,210719,21,5446,256,FOLIAR 5,Fertilizante,13.0
5,12,3942,012,2019-07-15,210719,32197,2019-02-28,2020-03-27,210719,21,5446,256,FOLIAR 6,Fertilizante,28.0
6,15,3998,014,2019-07-29,210719,32197,2019-02-28,2020-03-27,210719,21,5446,256,FOLIAR 7,Fertilizante,14.0
7,17,4116,015,2019-08-15,210719,32197,2019-02-28,2020-03-27,210719,21,5446,256,FOLIAR 8,Fertilizante,17.0
8,18,4104,017,2019-08-30,210719,32197,2019-02-28,2020-03-27,210719,21,5446,256,FOLIAR 9,Fertilizante,15.0
9,20,4159,018,2019-09-10,210719,32197,2019-02-28,2020-03-27,210719,21,5446,256,FOLIAR 10,Fertilizante,11.0


In [3]:
df = pd.read_excel("prueba_query_memoria.xlsx")

In [5]:
df.groupby(["blocknumber","area","plantcant","finiciosiembra","finduccion"],dropna=False)["diff"].mean()

blocknumber  area  plantcant  finiciosiembra  finduccion
290620       1848  8090       2020-05-27      NaN           18.833333
291020       3045  15181      2020-05-22      NaN           18.833333
291320       8166  33450      2020-05-22      NaN           18.833333
291520       4446  19050      2020-05-22      NaN           18.833333
291620       4224  21268      2020-05-26      NaN           23.500000
Name: diff, dtype: float64

In [3]:
categorias_de_aplicacion = pd.read_excel("relacion_insumos_principales_categorias.xlsx")

In [21]:
peso_planta = pd.read_excel("peso_planta.xlsx",dtype={"bloque":str})
peso_planta.sort_values(by=["bloque","edad"],inplace=True)


In [25]:
import plotly.express as px
df = pd.DataFrame(columns=["edad","promedio"])
fig = px.scatter(df, x="edad", y="promedio")

In [15]:
homologacion = categorias_de_aplicacion.groupby("codigo")["categorias_por_insumo"].apply(lambda x: ','.join(x)).reset_index()

In [17]:
homologacion[homologacion['categorias_por_insumo'].str.contains("herbicida")]

,codigo,categorias_por_insumo
0,001,herbicida
8,010,herbicida
22,032.V10,herbicida


In [29]:
categorias_de_aplicacion.iat[0,-1]

'herbicida'

In [9]:
categorias_de_aplicacion.iat[10,-1].split("'")

['{', 'herbicida', '}']

In [36]:
tooltip = [
        {
            column: {'value': str(value), 'type': 'markdown'}
            for column, value in row.items()
        } for row in nutricion_preforza_pc[["formula"]].to_dict('rows')
    ]

In [42]:
tooltip[3]

{'formula': {'value': 'FF-01', 'type': 'markdown'}}

In [75]:
insumos = pd.read_excel("insumos_por_aplicacion.xlsx")
insumos["agrupacion"] = insumos.groupby("id_formula")["Descripcion"].transform(lambda x: "\n".join(x))
insumos.drop_duplicates(subset=["id_formula"],inplace=True)
insumos["tooltip"]="Descripción aplicación: " + insumos.descripcion_formula + "\n"+insumos.agrupacion
diccionario_insumos = insumos.set_index("id_formula")["tooltip"].to_dict()

In [2]:
calendario_plan = pd.read_excel("calendario_aplicaciones.xlsx")
calendario_plan.drop(["y"],axis=1,inplace=True)
aplicaciones = pd.read_excel("entregable1.xlsx",sheet_name="data")

In [4]:
#Extraer fechas de inicio
fecha_inicio = aplicaciones.sort_values(by=["bloque","fecha_aplicacion"]).drop_duplicates(subset=["bloque"],keep='first')
dicc_fecha_inicio = {}
for index, row in fecha_inicio.iterrows():
    dicc_fecha_inicio[row["bloque"]] = row["fecha_aplicacion"]

In [35]:
aplicaciones_unicas = aplicaciones.sort_values(by=["bloque","fecha_aplicacion"]).drop_duplicates(subset=["bloque","homologación"],keep='first')[["fecha_aplicacion","bloque","homologación"]]

In [54]:
dicc_plan_aplicaciones = {}
for bloque in dicc_fecha_inicio.keys():

    fecha_inicial = dicc_fecha_inicio[bloque]
    calendario_temp = calendario_plan.copy()
    calendario_temp["bloque"]=bloque
    calendario_temp["fechas"]=calendario_temp["dias"].apply(lambda x: fecha_inicial +pd.Timedelta(days=x))
    ejecucion_temp = aplicaciones_unicas.query("bloque==@bloque").drop(["bloque"],axis=1)
    ejecucion_temp.columns = ["fecha_ejecucion","aplicacion"]
    calendario_temp = calendario_temp.merge(ejecucion_temp,how='left',on="aplicacion")
    calendario_temp["ejecucion-plan"]= calendario_temp["fecha_ejecucion"]-calendario_temp["fechas"]
    dicc_plan_aplicaciones[bloque] = calendario_temp.copy()

file_to_write = open("seguimiento_aplicaciones.pkl", "wb")
pickle.dump(dicc_plan_aplicaciones, file_to_write)

In [57]:
type(object_file)

dict

In [82]:
file = open("seguimiento_aplicaciones.pkl",'rb')
object_file = pickle.load(file)
file.close()
fechas_pendientes = pd.concat(object_file.values()).query('fecha_ejecucion in [@pd.NaT]')[["aplicacion","bloque","fechas"]]
fechas_pendientes["retraso"]=pd.Timestamp.today() - fechas_pendientes["fechas"]
fechas_pendientes["retraso"] = fechas_pendientes.retraso.dt.days
fechas_pendientes.query("retraso>0",inplace=True)
fechas_pendientes.sort_values(by="retraso",ascending=False,inplace=True)

In [83]:
fechas_pendientes

,aplicacion,bloque,fechas,retraso
8,Edafico 1,10619,2020-03-22,206
4,Protección 2,10120,2020-05-06,161
4,Protección 2,10419,2020-05-06,161
4,Protección 2,10220,2020-05-06,161
4,Protección 2,10519,2020-05-06,161
4,Protección 2,10320,2020-05-06,161
8,Edafico 1,10220,2020-05-31,136
8,Edafico 1,10519,2020-05-31,136
8,Edafico 1,10120,2020-05-31,136
8,Edafico 1,10419,2020-05-31,136


In [67]:
pd.Timestamp.today()

Timestamp('2020-10-14 08:29:18.430450')